Name: Gökçe Uludoğan

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


In [558]:
import numpy as np
import random
import functools
import re
import itertools
import math
t_matrix = np.matrix(T)

In [84]:
def generate_string(N):
    generated_string = '.'
    for i in range(N):
        cdf = np.cumsum(np.asarray(t_matrix[letter2idx[generated_string[-1]]], dtype=float))
        rand_value = random.uniform(0, cdf[-2])
        letter_ind = np.where( cdf > rand_value )[0].min()
        generated_string +=  alphabet[letter_ind]
    return generated_string[1:]    
generate_string(5)            

'indic'

In [442]:
def fill_string(string_incomplete, allow_space=False):
    string_incomplete = '.' + string_incomplete
    it = re.finditer(r"[_?]+", string_incomplete)
    missing_sequences = [(m.start(0), m.end()) for m in it]
    missing_starts = np.array([x for x, y in missing_sequences])
    missing_ends = np.array([y for x, y in missing_sequences])
    generated_string = ''
    str_len = len(string_incomplete)
    missing = False
    for i in range(str_len):
        #print("current", string_incomplete[i])
        if i in missing_ends:
            missing = False
        elif i in missing_starts:
            missing = True
        if missing:
            #print(i)
            left_char_ind = letter2idx[generated_string[i-1]]
            left_prob = t_matrix[left_char_ind]
            end_ind = min(missing_ends[missing_ends > i])
            right_char_ind = letter2idx[string_incomplete[end_ind] if len(string_incomplete) > end_ind else '.']
            right_prob = t_matrix[:, right_char_ind]
            no_mult = end_ind - i - 1
            if no_mult == 0:
                missing_prob = np.ones(t_matrix.shape)
            else:    
                missing_prob = functools.reduce(np.dot, [t_matrix] * (no_mult))
            if len(string_incomplete) > end_ind:    
                prob = np.multiply(right_prob.T , left_prob) / missing_prob[left_char_ind, right_char_ind]
            else:
                prob = np.divide(left_prob, missing_prob[left_char_ind])
            cdf = np.cumsum(np.asarray(prob, dtype=float))
            # allow '.' for filling missing char or not. 
            if allow_space:
                last_ind = -1
            else:
                last_ind = -2
            rand_value = random.uniform(0, cdf[last_ind])
            letter_ind = np.where( cdf > rand_value )[0].min()
            generated_string +=  alphabet[letter_ind]
        else:
            generated_string += string_incomplete[i]

    return generated_string[1:]

In [512]:
for string_incomplete in test_strings:
    print("String with missing chars:\t\t", string_incomplete)
    print("Generated string without spaces:\t", fill_string(string_incomplete))
    print("Generated string with spaces:\t\t", fill_string(string_incomplete, allow_space=True))

String with missing chars:		 th__br__n.f_x.
Generated string without spaces:	 thambrain.fix.
Generated string with spaces:		 tho.brorn.fex.
String with missing chars:		 _u_st__n_.to_be._nsw_r__
Generated string without spaces:	 ouisturno.tombe.answerra
Generated string with spaces:		 cu.stadng.to.be.answhrwh
String with missing chars:		 i__at_._a_h_n_._e_r_i_g
Generated string without spaces:	 imbato.hathand.retrning
Generated string with spaces:		 il.ath.cashind.te.rding
String with missing chars:		 q___t.___z._____t.__.___.__.
Generated string without spaces:	 quect.rirz.ouasst.an.ate.sh.
Generated string with spaces:		 qun.t.antz..s.ist.dg.ty..ou.


In [572]:
def most_likely(string_incomplete, allow_space=False):
    t_matrix[t_matrix == 0] = 1e-6
    if allow_space:
        letter_list = alphabet
    else:
        letter_list = alphabet[:-1]
    string_incomplete = '.' + string_incomplete
    it = re.finditer(r"[_?]+", string_incomplete)
    missing_sequences = [(m.start(0), m.end()) for m in it]
    total_likelihood = 0.0
    most_likely_str = ''
    last = 0
    for missing_start, missing_end in missing_sequences:
        most_likely_str += string_incomplete[last:missing_start] 
        last = missing_end
        no_of_char = missing_end - missing_start
        left_char = string_incomplete[missing_start-1]
        right_char = string_incomplete[missing_end] if missing_end < len(string_incomplete) else ''
        seq_probs = {}
        for sequence in itertools.product(letter_list, repeat=no_of_char):
            seq_total = left_char + ''.join(sequence) + right_char
            probability = sum([math.log(t_matrix[letter2idx[char], letter2idx[seq_total[index + 1]]]) for index, char in enumerate(seq_total) if index < len(seq_total) - 1])
            #print(sequence, str(probability))
            seq_probs[''.join(sequence)] = probability 
        max_likelihood = sorted(seq_probs.items(), key=lambda item: item[1], reverse=True)[0] 
        total_likelihood += max_likelihood[1]
        most_likely_str += max_likelihood[0]
    most_likely_str += string_incomplete[last:]    
    return most_likely_str[1:], total_likelihood

In [575]:
for string in test_strings:
    print('Test string\t', string)
    most_likely_str, prob = most_likely(string, allow_space=True)
    print('Most likely\t', most_likely_str)
    print('Log probability\t', str(prob))
    print('-------------------------------')

Test string	 th__br__n.f_x.
Most likely	 the.br.an.fex.
Log probability	 -17.141135970996658
-------------------------------
Test string	 _u_st__n_.to_be._nsw_r__
Most likely	 oursthend.to.be.answere.
Log probability	 -31.549808199783044
-------------------------------
Test string	 i__at_._a_h_n_._e_r_i_g
Most likely	 in.ath.wathend.he.r.ing
Log probability	 -37.44445564208814
-------------------------------
Test string	 q___t.___z._____t.__.___.__.
Most likely	 qus.t.herz.thed.t.he.the.he.
Log probability	 -38.50641808973078
-------------------------------


### Improvements

* 

*

*